In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import gc

!ls ../input/indoor-location-navigation

In [ ]:
floor_map = {
             "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8, "F10":9,
             "1F":0, "2F": 1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6,"8F":7, "9F":8, "10F":9,
            
            "B":0, "B1":-1, "B2":-2, "B3":-3, "B4":-4, "B5":-5, "B6":-6, "B7":-7, "B8":-8, "B9":-9,
                   "1B":-1, "2B":-2, "3B":-3, "4B":-4, "5B":-5, "6B":-6, "7B":-7, "8B":-8, "9B":-9,
    
            "G":0, "L2": 1, "L3":2, "L4":3, "L5":4, "L6":5, "L7":6,"L8":7, "L9":8, "L10":9,
            
            "L1":0, "LG1": 1, "LG2":2, "LM":3, "M":4, "P1":0, "P2":1,
            }

floor_map

In [ ]:
trainfiles = glob.glob('../input/indoor-location-navigation/train/*/*/*')
testfiles = glob.glob('../input/indoor-location-navigation/test/*')
len(trainfiles), len(testfiles), trainfiles[0], testfiles[0]

In [ ]:
floor, floor_count = np.unique( [ f.split('/')[-2] for f in trainfiles ], return_counts=True )
floor, floor_count

In [ ]:
!head ../input/indoor-location-navigation/train/5cd56c0ce2acfd2d33b6ab27/B1/5d09a625bd54340008acddb9.txt

In [ ]:
def load_dict( filename ):
    with open(filename) as f:
        lines = f.readlines()

    proc = {}
    SiteID = ''
    count = 0
    for l in lines:
        l = l.replace('\n','')
        
        if l[0]!='#':
            val = l.split('\t')
            
            ts = val[0]
            if ts not in proc:
                proc[ts] = {}
                proc[ts]['wp0'] = np.nan
                proc[ts]['wp1'] = np.nan
                proc[ts]['wifi0'] = np.nan
                proc[ts]['wifi1'] = np.nan
                proc[ts]['ts'] = ts
                proc[ts]['SiteID'] = SiteID

            if val[1] == 'TYPE_WAYPOINT':
                proc[ts]['wp0']= float(val[2])
                proc[ts]['wp1']= float(val[3])

            elif val[1] == 'TYPE_WIFI':
                proc[ts]['wifi0']= val[2]
                proc[ts]['wifi1']= val[2]

            count+=1
                
                
        else:
            val = l.split('\t')
            #print(val)
            if len(val)>=2:
                if val[1][:7] == 'SiteID:':
                    SiteID = val[1].split(':')[-1]

    return proc

In [ ]:
!ls ../input/gibatewifidataset1

In [ ]:
# TRAIN = []
# for fn in tqdm(trainfiles):
#     path = load_dict( fn )
#     #print( len(path), fn )

#     dt = pd.DataFrame( {
#         'SiteID':[path[f]['SiteID'] for f in path],
#         'ts':[path[f]['ts'] for f in path],
#         'wifi0':[path[f]['wifi0'] for f in path],
#         'wifi1':[path[f]['wifi1'] for f in path],
#         'wp0':[path[f]['wp0'] for f in path],
#         'wp1':[path[f]['wp1'] for f in path],
#     } )

#     dt['floor'] = fn.split('/')[-2]

#     dt['wifi0'] = dt['wifi0'].fillna(method='ffill')
#     dt['wifi1'] = dt['wifi1'].fillna(method='ffill')
#     dt['wifi0'] = dt['wifi0'].fillna(method='bfill')
#     dt['wifi1'] = dt['wifi1'].fillna(method='bfill')

#     dt['wp0'] = dt['wp0'].interpolate()
#     dt['wp1'] = dt['wp1'].interpolate()
    
#     grp = dt.groupby( ['wifi0','wifi1','floor'] )['wp0','wp1'].agg('mean').reset_index()
#     grp['SiteID'] = dt['SiteID'].values[0]
    
#     TRAIN.append(grp)
# gc.collect()

# TRAIN = pd.concat( TRAIN, sort=False )
# TRAIN.to_csv('train_files.csv', index=False)
# TRAIN.shape

TRAIN = pd.read_csv( '../input/gibatewifidataset1/train_files.csv' )

In [ ]:
TRAIN.head()

In [ ]:
!head -n 20 ../input/indoor-location-navigation/test/52ad8c760ff9978d0949deed.txt

In [ ]:
!ls ../input/indoor-location-navigation

In [ ]:
sub = pd.read_csv( '../input/indoor-location-navigation/sample_submission.csv' )
print( sub.shape )
sub['SiteID'] = sub['site_path_timestamp'].apply( lambda x: x.split('_')[0] )
sub['path'] = sub['site_path_timestamp'].apply( lambda x: x.split('_')[1] )
sub['ts'] = sub['site_path_timestamp'].apply( lambda x: x.split('_')[2] )
#sub.iloc[1,0]

print( sub.nunique() )

sub.head()

In [ ]:
len(testfiles), testfiles[0]

In [ ]:
TEST = []
for fn in tqdm(testfiles):
    path = load_dict( fn )
    #print( len(path), fn )

    dt = pd.DataFrame( {
        'SiteID':[path[f]['SiteID'] for f in path],
        'ts':[path[f]['ts'] for f in path],
        'wifi0':[path[f]['wifi0'] for f in path],
        'wifi1':[path[f]['wifi1'] for f in path],
        'wp0':[path[f]['wp0'] for f in path],
        'wp1':[path[f]['wp1'] for f in path],
    } )

    dt['floor'] = fn.split('/')[-2]

    dt['wifi0'] = dt['wifi0'].fillna(method='ffill')
    dt['wifi1'] = dt['wifi1'].fillna(method='ffill')
    dt['wifi0'] = dt['wifi0'].fillna(method='bfill')
    dt['wifi1'] = dt['wifi1'].fillna(method='bfill')

    dt['wp0'] = dt['wp0'].interpolate()
    dt['wp1'] = dt['wp1'].interpolate()
    
    dt['path'] = fn.split('/')[-1].split('.')[0]
    
    TEST.append(dt)
gc.collect()

TEST = pd.concat( TEST, sort=False )
TEST.shape

In [ ]:
TEST['site_path_timestamp'] = TEST['SiteID'] + '_' + TEST['path'] + '_' + TEST['ts']
TEST['istest'] = 0
TEST.head()

In [ ]:
sub['SiteID'] = sub['site_path_timestamp'].apply(lambda x: x.split('_')[0] )
sub['path'] = sub['site_path_timestamp'].apply(lambda x: x.split('_')[1] )
sub['ts'] = sub['site_path_timestamp'].apply(lambda x: x.split('_')[2] )
sub['istest'] = 1
sub.head()

In [ ]:
print(TEST.shape)
TESTall = pd.concat( (TEST,sub), sort=False )
gc.collect()
print(TESTall.shape)
TESTall.head()

In [ ]:
TESTall = TESTall.sort_values( ['site_path_timestamp'] )
TESTall['wifi0'] = TESTall['wifi0'].fillna(method='bfill')
TESTall['wifi0'] = TESTall['wifi0'].fillna(method='ffill')
TESTall['wifi1'] = TESTall['wifi1'].fillna(method='bfill')
TESTall['wifi1'] = TESTall['wifi1'].fillna(method='ffill')
TESTall.head()

In [ ]:
TRAIN.head()

In [ ]:
TRAIN.nunique()

In [ ]:
TRAIN['fold'] = TRAIN['SiteID'].apply( lambda x: hash(x)%10 )
TRAIN['fold'].value_counts()

In [ ]:
TRAIN['lvl'] = TRAIN['floor'].map(floor_map)
TRAIN.head()

In [ ]:
train = TRAIN.loc[ TRAIN.fold!=0 ].copy()
valid = TRAIN.loc[ TRAIN.fold==0 ].copy()

In [ ]:
dt0 = train.groupby(['wifi0'])['wp0'].agg('mean').reset_index()
dt1 = train.groupby(['wifi0'])['wp1'].agg('mean').reset_index()
dt2 = train.groupby(['wifi0'])['lvl'].agg('mean').reset_index()
dt0, dt1, dt2

In [ ]:
valid['yx'] = valid[['wifi0']].merge( dt0, on=['wifi0'], how='left' )['wp0'].values
valid['yy'] = valid[['wifi0']].merge( dt1, on=['wifi0'], how='left' )['wp1'].values
valid['ylvl'] = valid[['wifi0']].merge( dt2, on=['wifi0'], how='left' )['lvl'].values

valid['yx'].isnull().mean(), valid['yy'].isnull().mean()

In [ ]:
# the metric used in this competition
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

In [ ]:
valid = valid.loc[ valid.yx.notnull() ].copy()
print( valid.shape )
valid.head()

In [ ]:
comp_metric( valid.yx, valid.yy, valid.ylvl, valid.wp0, valid.wp1, valid.lvl  )

In [ ]:
dt0 = TRAIN.groupby(['wifi0'])['wp0'].agg('mean').reset_index()
dt1 = TRAIN.groupby(['wifi0'])['wp1'].agg('mean').reset_index()
dt2 = TRAIN.groupby(['wifi0'])['lvl'].agg('mean').reset_index()

TESTall['x'] = TESTall[['wifi0']].merge( dt0, on=['wifi0'], how='left' )['wp0'].values
TESTall['y'] = TESTall[['wifi0']].merge( dt1, on=['wifi0'], how='left' )['wp1'].values
TESTall['floor'] = TESTall[['wifi0']].merge( dt2, on=['wifi0'], how='left' )['lvl'].values
TESTall.head()

In [ ]:
TESTall['x'] = TESTall['x'].interpolate()
TESTall['x'] = TESTall['x'].fillna( method='bfill' )
TESTall['x'] = TESTall['x'].fillna( method='ffill' )

TESTall['y'] = TESTall['y'].interpolate()
TESTall['y'] = TESTall['y'].fillna( method='bfill' )
TESTall['y'] = TESTall['y'].fillna( method='ffill' )

TESTall['floor'] = TESTall['floor'].interpolate()
TESTall['floor'] = TESTall['floor'].fillna( method='bfill' )
TESTall['floor'] = TESTall['floor'].fillna( method='ffill' )
TESTall['floor'] = TESTall['floor'].round().astype(np.int32)

TESTall.head()

In [ ]:
sub = TESTall.loc[ TESTall.istest==1, ['site_path_timestamp','x','y','floor'] ]
sub.shape

In [ ]:
sub.head()

In [ ]:
sub.to_csv( 'submission.csv', index=False )